# Import thư viện

In [18]:
import requests
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

In [ ]:
# ------------------- Thư mục lưu -------------------
OUT = Path(r"C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw")
OUT.mkdir(parents=True, exist_ok=True)


In [ ]:

# ------------------- Yahoo Chart API -------------------
def yahoo_chart(symbol: str, start_date="2015-01-01", interval="1d", close_only=True) -> pd.DataFrame:
    """
    Crawl dữ liệu thô từ Yahoo Finance chart API.
    Trả về DataFrame gồm giá theo ngày (close-only hoặc full OHLC).
    """
    # 1️⃣ Tính thời gian epoch
    dt1 = datetime.fromisoformat(start_date)
    period1 = int(dt1.replace(tzinfo=timezone.utc).timestamp())
    period2 = int(datetime.now(tz=timezone.utc).timestamp())

    # 2️⃣ Gửi request đến API
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"
    params = {
        "period1": period1,
        "period2": period2,
        "interval": interval,
        "includePrePost": "true",
        "events": "div,splits",
    }
    r = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
    r.raise_for_status()

    # 3️⃣ Parse dữ liệu JSON → DataFrame
    result = r.json().get("chart", {}).get("result")
    if not result:
        raise RuntimeError("Không có dữ liệu trả về từ Yahoo API.")
    node = result[0]
    ts = node.get("timestamp", [])
    ind = node.get("indicators", {})
    quote = (ind.get("quote") or [{}])[0]
    adj   = (ind.get("adjclose") or [{}])[0]

    # 4️⃣ Chuyển sang DataFrame
    df = pd.DataFrame({
        "datetime_utc": [datetime.fromtimestamp(t, tz=timezone.utc) for t in ts],
        "close": quote.get("close"),
    })
    if not close_only:
        df["open"] = quote.get("open")
        df["high"] = quote.get("high")
        df["low"] = quote.get("low")
        df["volume"] = quote.get("volume")
        df["adjclose"] = adj.get("adjclose")

    return df

In [20]:
# ------------------- Main Crawl -------------------
if __name__ == "__main__":
    symbols = [
        ("VND=X", "2015-01-01", "1d"),
        ("BZ=F", "2015-01-01", "1d"),
       
    ]

    for sym, start, interval in symbols:
        try:
            df = yahoo_chart(sym, start_date=start, interval=interval, close_only=True)
            out_path = OUT / f"{sym.replace('=','_').replace('^','')}_{interval}.csv"
            df.to_csv(out_path, index=False, encoding="utf-8-sig")
            print(f"✅ {sym} ({interval}) -> {len(df)} dòng | Lưu: {out_path}")
        except Exception as e:
            print(f"❌ Lỗi {sym}: {e}")

✅ VND=X (1d) -> 2816 dòng | Lưu: C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw\VND_X_1d.csv
✅ BZ=F (1d) -> 2719 dòng | Lưu: C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw\BZ_F_1d.csv
